In [67]:
# Libraries

#Connect to mysql
import mysql.connector

#Operating system
import os
import sys
from pathlib import Path
import subprocess
from datetime import timedelta

#Transcription & subtitles
import whisper
import stable_whisper

#Translation
import translators as ts
import translators.server as tss
import textwrap

#Pandas
import pandas as pd


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [14]:
def transcribe(filepath):
    """
    With this function we can transcribe all the texts from a video/audio and also the subtitles
    
    Keyword arguments:
    argument -- filepath:the file to transcribe
    Return: No return (insert in mysql and create 2 text files -temporary-)
    """
    
    video = os.path.split(os.path.abspath(Path(filepath)))
    name = video[1].split(sep='.')

    result_sql = insert_data_sql('video', '', '', filepath)
    videoid = result_sql.lastrowid
    
    # speech transcription
    
    model = whisper.load_model("base.en",device='cpu')
    #model = stable_whisper.load_model('base')
    result = model.transcribe(filepath)
    
    with open(Path(video[0]+"/"+name[0]+"_transcription.txt"), "w+") as f:
        f.write(result["text"])

    result_sql = insert_data_sql('transcription', videoid, 'en', result['text'])


    # subtitles 
    stable_whisper.results_to_sentence_srt(result, video[0]+"/"+name[0]+"_subtitles.srt")

    text_subtitles = open(video[0]+"/"+name[0]+"_subtitles.srt").read()

    result_sql = insert_data_sql('subtitle', videoid, 'en', text_subtitles)
    


In [ ]:
#transcribe('/home/roque/01. IronHack/00. Data Analytics/01. Course/04. Week 2 - Day 1/recordings/GMT20220913-171210_Recording_1920x1080.mp4')
#transcribe('/home/roque/01. IronHack/00. Data Analytics/01. Course/02. Week 1 - Day 2/Recordings/GMT20220908-163034_Recording_1920x1080.mp4')
#transcribe('/home/roque/01. IronHack/00. Data Analytics/01. Course/01. Week 1 - Day 3/Recordings/GMT20220910-082751_Recording_1920x1080.mp4')
#transcribe('/home/roque/01. IronHack/00. Data Analytics/01. Course/01. Week 2 - Day 1/Recordings/GMT20220913-171210_Recording_1920x1080.mp4')
#transcribe('/home/roque/01. IronHack/00. Data Analytics/01. Course/01. Week 2 - Day 2/Recordings/GMT20220915-163321_Recording_1920x1080.mp4')


In [ ]:
# To use the subtitles, can use: vlc test.mp4 --sub-file test2.srt
#subprocess.call(['vlc', '/home/roque/01. IronHack/00. Data Analytics/01. Course/03. Week 1 - Day 3/recordings/GMT20220910-082751_Recording_1920x1080.mp4', '--sub-file', '/home/roque/01. IronHack/00. Data Analytics/01. Course/03. Week 1 - Day 3/recordings/GMT20220910-082751_Recording_1920x1080_subtitles.srt', '--start-time', '3120'])

In [ ]:
def video_player(_videoid, _langid, _position = 0):
    conn = connection_ddbb()
    cursor = conn.cursor() 

    if conn.is_connected:
        #Configuration table
        query = """SELECT video_player, temp_directory
                     FROM ironrep.configuration 
                     LIMIT 1;"""

        cursor.execute(query)
        conf_table = cursor.fetchall()
        conf_df = pd.DataFrame(conf_table)
        conf_df.columns = [i[0] for i in cursor.description]

        #subtitles

        #first read the subtitles from the database
        query = """SELECT subtitles
                     FROM ironrep.subtitles"""
        cursor.execute(query)
        subt_table = cursor.fetchall()

In [64]:
conn = connection_ddbb()
cursor = conn.cursor() 
query = """SELECT video_player, temp_directory
                     FROM ironrep.configuration 
                     LIMIT 1;"""

cursor.execute(query)
conf_table = cursor.fetchall()
conf_df = pd.DataFrame(conf_table)
conf_df.columns = [i[0] for i in cursor.description]

NameError: name 'pd' is not defined

In [63]:
conf_table

[('vlc {videoparam} --sub-file {subtitleparam} --start-time {positionparam}',
  '/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/data/')]

In [4]:
def translate_from_en(text_en,to_language):
    """FUNCTION TO TRANSLATE THE TEXT FROM ENGLISH TO ANY LANGUAGE
    
    Keyword arguments:
    text_en: text in english
    to_language: the code of the language to translate (es,zh, ...) 
    Return: text translated
    """
    
    if (to_language==''):
        display('Need to include the language to translate.')
    else:
        ts.translators_pool

        text_translated = ''
        
        from_language = 'en'
            
        
        if (len(text_en)>2000):
            textsplited_to = []
            text_splited = textwrap.wrap(text_en, 2000, break_long_words=False)
            for line in text_splited:
                textsplited_to.append(tss.google(line, from_language, to_language))
            text_translated = ' '.join(textsplited_to)
        else:
            try:
                text_translated = tss.google(text_en, from_language, to_language)
            except:
                try:
                    text_translated = ts.translate_text(text_en, from_language, to_language)
                except:
                    text_translated = text_en

    return text_translated

In [ ]:
translate_from_en('hello','zh')

In [57]:
def connection_ddbb():
    """Function to create the conection to the data base
    
    Keyword arguments:    
    Return: connection objet
    """
       

    secrets={}
    #secrets_file = open('secrets.txt','r') #Had errors using this within a virtual environment
    secrets_file = os.fdopen(os.open('secrets.txt', os.O_RDONLY))
    for line in secrets_file:
        (key, val) = line.replace('\n','').split("|")
        secrets[key] = val    

    #Conection to mysql

    conn = mysql.connector.connect(user=secrets['user'],
                            password=secrets['pass'],
                            host=secrets['server'])
    
    return conn

In [60]:
def create_ddbb():
    """Function to create the data base
    
    Keyword arguments:    
    Return: None
    """
    
    conn = connection_ddbb()

    cursor = conn.cursor()

    #Creating schema in mysql

    if conn.is_connected():
        print('Connection open')        
        
        print('Creating database if necessary...')
        
        query = ('CREATE DATABASE IF NOT EXISTS ironrep')
        
        cursor.execute(query)
        conn.commit()
        
        query = ('USE ironrep')
        
        cursor.execute(query)
        conn.commit()
        
        print('Database created if necessary...')

        print('Creating tables if necessary...')

        #Configuration        
        
        query = ("""CREATE TABLE IF NOT EXISTS ironrep.configuration (  
                        id enum('1') PRIMARY KEY NOT NULL,
                        temp_directory  nvarchar(250),
                        video_player nvarchar(250),
                        languages_subtitles nvarchar(250) COMMENT 'List of langages codes separated by commas')""")

        cursor.execute(query)    
        conn.commit()

        #Default values
        query = ("""REPLACE INTO ironrep.configuration (temp_directory,video_player,languages_subtitles)
                        VALUES (%s,%s,%s)""")
        val = (str(secrets['temp_dir']),str(secrets['video_play']),str(secrets['lang_subt']))
        """val = (str('/home/roque/01. IronHack/00. Data Analytics/01. Course/63. Week 23 - Day 3/git/final-project-bootcamp/data/'),
                str('vlc {videoparam} --sub-file {subtitleparam} --start-time {positionparam}'),
                str('es,pt,it,zh,de,hi'))"""
        cursor.execute(query,val)
        conn.commit()

        #Videos

        query = ("""CREATE TABLE IF NOT EXISTS ironrep.videos (
                        id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
                        video nvarchar(250),
                        video_path nvarchar(250)
                    )"""
        )

        cursor.execute(query)    
        conn.commit()

        #Transcriptions

        query = ("""CREATE TABLE IF NOT EXISTS ironrep.transcriptions (
                        id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
                        videoid INT NOT NULL,
                        languageid nvarchar(5),
                        transcription mediumtext
                    )"""
        )

        cursor.execute(query)    
        conn.commit()

        #Summaries

        query = ("""CREATE TABLE IF NOT EXISTS ironrep.summaries (
                        videoid INT NOT NULL,
                        languageid nvarchar(5),
                        summary nvarchar(260)                   
                    )"""
        )

        cursor.execute(query)    
        conn.commit()

        #Subtitles

        query = ("""CREATE TABLE IF NOT EXISTS ironrep.subtitles (
                        id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
                        videoid INT NOT NULL,
                        languageid nvarchar(5),
                        subtitles mediumtext                   
                    )"""
        )

        cursor.execute(query)    
        conn.commit()

        #Keywords

        query = ("""CREATE TABLE IF NOT EXISTS ironrep.keywords (
                        id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
                        videoid INT NOT NULL,
                        languageid nvarchar(5),
                        keywords nvarchar(250)                   
                    )"""
        )

        cursor.execute(query)    
        conn.commit()

        print('Tables created if necessary...')
    else:
        print('Error connecting')


    

In [61]:
def insert_data_sql(_table, _videoid, _langid, _text):
    """Function to insert data in the database
    
    Keyword arguments:
    _table: the object where insert ['video','transcription','subtitle','summary','keywords']
    _videoid: the id of the video
    _langid: the id of the language
    _text: value to insert
    Return: cursor
    """
    
    #Connection to the ddbb

    conn = connection_ddbb()

    if conn.is_connected():
        query = ''
        val = []

        if (_table == 'video'):                 
            vid_path = os.path.split(os.path.abspath(Path(_text)))

            query = """INSERT INTO ironrep.videos(video,video_path)
                        VALUES (%s,%s)"""
            val = [vid_path[1].split('.')[0],_text]
        elif (_table == 'transcription'):
            query = """INSERT INTO ironrep.transcriptions(videoid,languageid,transcription)
                        VALUES (%s,%s,%s)"""
            val = [int(_videoid),_langid,_text]
        elif (_table == 'subtitle'):
            query = """INSERT INTO ironrep.subtitles(videoid,languageid,subtitles)
                        VALUES (%s,%s,%s)"""
            val = [int(_videoid),_langid,_text]
        elif (_table == 'summary'):
            query = """INSERT INTO ironrep.summary(videoid,languageid,summary)
                        VALUES (%s,%s,%s)"""
            val = [int(_videoid),_langid,_text]
        elif (_table == 'keywords'):
            query = """INSERT INTO ironrep.keywords(videoid,languageid,keywords)
                        VALUES (%s,%s,%s)"""
            val = [int(_videoid),_langid,_text]
        else:
            return 'none'

        if (query != ''):      
            cursor.execute(query,val)
            conn.commit()  
            return cursor

In [7]:
# Summary
# TODO: Try with a larger text

# python -m spacy download en_core_web_sm #eficency
# python -m spacy download en_core_web_trf #accuracy

import spacy
import en_core_web_trf
#import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
import spacy_transformers
from string import punctuation
from heapq import nlargest

def summarize(text, per):
    #nlp = spacy.load('en_core_web_trf')
    nlp = en_core_web_trf.load()
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [ ]:
summarize(result['text'], 0.10)

In [10]:
+# Keywords

#RAKE
from rake_nltk import Rake
rake = Rake()
rake.extract_keywords_from_text(result['text'])
extracted_keyword = rake.get_ranked_phrases()
extracted_keyword

['like one main one main one main suggestion',
 'whole thing like drives like applies force',
 '30 grammets per hour speed limits',
 'insanely strong tech tech right behind',
 'pretty high cpc cost per click',
 'new race fund group b',
 'analysis like drives towards towards',
 'passengers involved involving car accident obviously',
 'strong hypothesis like super well laid',
 'getting like disparate data together like',
 'regular two bars one one next',
 'pretty intense machine learning technique',
 'specific works like vhvac shit',
 'thousand dollar investment per trade',
 'barcelona like 50 speed limit pro',
 'available like entire home apartment',
 'data set time seven parameters',
 'create something called budget persona',
 '46 per 46 values',
 'transcutaneous vagus nerve stimulation',
 'organic email marketing also influencer',
 'entire home apartment type cost',
 'think maybe 10 shapes something like',
 'normally find hypothesis test hypothesis find',
 'great like storytelling sto